## Sorting the images and labels

In [1]:
import cv2
import numpy as np
import pandas as pd
import glob, os, re, time, sys, math, ast, json

print('Importing Dependencies')

Importing Dependencies


### Generic functions for processing the data

In [3]:
def catalog_to_dataframe(line_list):
    res = [ast.literal_eval(x) for x in line_list] 
    dataframe = pd.DataFrame(res)
    dataframe = dataframe.set_index('_index')
    dataframe = dataframe.drop(['_session_id','_timestamp_ms', 'user/mode'], axis = 1)
    return dataframe

def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def make_dataframe(main_dir):
    catalog_files=sorted(glob.glob(main_dir), key=numericalSort)
    myDataFrame = pd.DataFrame()
    for i in range (0, len(catalog_files)):
        infile = open(catalog_files[i]) 
        lines = infile.readlines()
        df = catalog_to_dataframe(lines)
        myDataFrame = myDataFrame.append(df)
    return myDataFrame

def slicing(dataframe, r1, r2, string):
    if None not in (r1, r2):
        column = dataframe.loc[r1:r2,string]
    else:
        column = dataframe.loc[:,string]
    arr = column.values
    return arr
    
def steeing_throttle(n, s_t, f):
    nn = n.tolist()
    rounded_nn = [ '%.6f' % i for i in nn ]
    if s_t=='s':
        with open('steering_{}.json'.format(f), 'w', encoding='utf-8') as f:  # saving steering value as json file
            json.dump(nn, f, ensure_ascii=False, indent=4)
    if s_t=='t':
        with open('throttle_{}.json'.format(f), 'w', encoding='utf-8') as f:  # saving throttle value as json file
            json.dump(nn, f, ensure_ascii=False, indent=4)
    else:
        pass
    return rounded_nn

### Sorting labels from the directory

In [4]:
dir_1=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_1/*.catalog'
dir_2=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_2/*.catalog'
dir_3=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_3/*.catalog'
dir_4=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_4/*.catalog'
dir_5=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_5/*.catalog'
dir_6=r'C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_6/*.catalog'

df_1 = make_dataframe(dir_1)
df_2 = make_dataframe(dir_2)
df_3 = make_dataframe(dir_3)
df_4 = make_dataframe(dir_4)
df_5 = make_dataframe(dir_5)
df_6 = make_dataframe(dir_6)

#Steering 
as1=slicing(df_1, 264, 20472, 'user/angle')
as2=slicing(df_2, 565, len(df_2), 'user/angle')
as3=slicing(df_3, 218, len(df_3), 'user/angle')
as4=slicing(df_4, 0, 15433, 'user/angle')
as5=slicing(df_5, 0, len(df_5), 'user/angle')
as6=slicing(df_6, 0, 24340, 'user/angle')

#Throttle 
at1=slicing(df_1, 264, 20472, 'user/throttle')
at2=slicing(df_2, 565, len(df_2),'user/throttle')
at3=slicing(df_3, 218, len(df_3),'user/throttle')
at4=slicing(df_4, 0, 15433, 'user/throttle')
at5=slicing(df_5, 0, len(df_5),'user/throttle')
at6=slicing(df_6, 0, 24340, 'user/throttle')

n_steer=np.concatenate((as1, as2, as3, as4, as5, as6),axis=0)
n_throttle=np.concatenate((at1, at2, at3, at4, at5, at6),axis=0)

steering_list_new=steeing_throttle(n_steer, s_t='s', f='new')
throttle_list_new=steeing_throttle(n_throttle, s_t='t', f='new')

In [6]:
len(as1), len(as2), len(as3), len(as4), len(as5), len(as6)

(20209, 52861, 40889, 15434, 23625, 24341)

### Sorting images from the directory 

In [8]:
img_dir1="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_1/*.jpg"
img_dir2="C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_2/*.jpg"
img_dir3='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_3/*.jpg'
img_dir4='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_4/*.jpg'
img_dir5='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_5/*.jpg'
img_dir6='C:/Users/Nishanth Siegener/Downloads/Studienarbeit/Bumblebee/Data/new_data_6/*.jpg'

files_1=sorted(glob.glob(img_dir1), key=numericalSort)
files_2=sorted(glob.glob(img_dir2), key=numericalSort)
files_3=sorted(glob.glob(img_dir3), key=numericalSort)
files_4=sorted(glob.glob(img_dir4), key=numericalSort)
files_5=sorted(glob.glob(img_dir5), key=numericalSort)
files_6=sorted(glob.glob(img_dir6), key=numericalSort)

order_new = [files_1[264:20473], files_2[565:], files_3[218:], files_4[:15434], files_5, files_6[0:24341]]

len(order_new[0]), len(order_new[1]), len(order_new[2]), len(order_new[3]),len(order_new[4]), len(order_new[5]), len(order_new)

(20209, 52861, 40889, 15434, 23625, 24341, 6)

### Saving the images in sequences

In [9]:
m=0
for i in range(0,len(order_new)):
    for j in order_new[i]:
        img=cv2.imread(j)
        m+=1
        break
        cv2.imwrite('C:/path/to/directory/{}.jpg'.format(m), img) #create new folder to save each time to avoid confusion